In [195]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

## import pandas as pd

#### 나이브 베이즈(Naive Bayes)
- 베이즈 알고리즘: 미지의 사건의 확률을 결정하는 수학적 원칙
- 나이브라는 말을 앞에 붙인 이유는 특정 부류의 각 특징들은 서로 독립적이라는 가정을 내포하고 있기 때문
- 단어 기반의 이메일 분류등의 특정 과제에 있어 가장 효율적인 계산양을 자랑한다
- 최종 결과의 확률 예측을 위해 상당히 많은 속성을 동시에 고려해야하는 상황에서 가장 잘 작동한다 
- 최종 결과에 미미한 영향을 미치는 특징이라 하더라도 기용한 모든 단서를 활용한다

In [2]:
# 날씨, 온도에 따라 축구 경기 진행 여부 데이터 생성
weather = ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy',
           'Sunny', 'Overcast', 'Overcast', 'Rainy']
temperature = ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild']
play = ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']

In [3]:
df = pd.DataFrame({'weather': weather,
                  'temperature': temperature,
                  'play': play})
df

,weather,temperature,play
0,Sunny,Hot,No
1,Sunny,Hot,No
2,Overcast,Hot,Yes
3,Rainy,Mild,Yes
4,Rainy,Cool,Yes
5,Rainy,Cool,No
6,Overcast,Cool,Yes
7,Sunny,Mild,No
8,Sunny,Cool,Yes
9,Rainy,Mild,Yes


#### 문제 1. 날씨가 overcast, 기온이 Mild일 때 경기를 할 확률은?
- P(play=Yes|weather=Overcast, temperature=Mild) 
    = P(weather=Overcast, temperature=Mild|play=Yes) * P(play=Yes) / P(weather=Overcast, temperature=Mild)
- P(weather=Overcast, temperature=Mild|play=Yes) = P(weather=Overcast|play=Yes) * P(temperature=Mild|play=Yes)
- P(weather=Overcast, temperature=Mild) = P(weather=Overcast) * P(temperature=Mild)    

In [4]:
len(df), len(df.query('play == "Yes"')), len(df.query('weather == "Overcast"')), len(df.query('temperature == "Mild"'))

(14, 9, 4, 6)

In [5]:
len(df.query('weather == "Overcast" and play == "Yes"')), len(df.query('temperature == "Mild" and play == "Yes"'))

(4, 4)

사전 확률
- P(play=Yes) = 9 / 14
- P(weather=Overcast) =  4 / 14
- P(temperature=Mild) = 6 / 14

사후 확률
- P(weather=Overcast|play=Yes) = 4 / 9
- P(temperature=Mild|play=Yes) = 4 / 9

In [6]:
# 공식 대입
((4 / 9) * (4 / 9) * (9 / 14)) / ((4 / 14) * (6 / 14))

1.0370370370370372

#### 문제 1. 날씨가 overcast, 기온이 Mild일 때 경기를 하지 않을 확률은?
- P(play=No|weather=Overcast, temperature=Mild) 
    = P(weather=Overcast, temperature=Mild|play=No) * P(play=No) / P(weather=Overcast, temperature=Mild)
- P(weather=Overcast, temperature=Mild|play=No) = P(weather=Overcast|play=No) * P(temperature=Mild|play=No)
- P(weather=Overcast, temperature=Mild) = P(weather=Overcast) * P(temperature=Mild)    

In [7]:
len(df.query('play == "No"'))

5

In [8]:
len(df.query('weather == "Overcast" and play == "No"')), len(df.query('temperature == "Mild" and play == "No"'))

(0, 2)

사전 확률
- P(play=No) = 5 / 14
- P(weather=Overcast) =  4 / 14
- P(temperature=Mild) = 6 / 14

사후 확률
- P(weather=Overcast|play=No) = 0 / 5
- P(temperature=Mild|play=No) = 2 / 5


In [9]:
# 공식 대입
((0 / 5) * (2 / 5) * (5 / 14)) / ((4 / 14) * (6 / 14))

0.0

축구를 할 확률이 더 크기 때문에 날씨가 Overcast이고 기온이 Mild일 때는 축구를 할 것이라고 분류

### GaussianNB

#### LabelEncoder

In [10]:
# feature 데이터가 문자열이기 때문에 LabelEncoding으로 숫자로 바꿔준다
le_weather = LabelEncoder()
le_temperature = LabelEncoder()
weather_encoded = le_weather.fit_transform(df['weather'])
temperature_encoded = le_temperature.fit_transform(df['temperature'])

weather_encoded, temperature_encoded

(array([2, 2, 0, 1, 1, 1, 0, 2, 2, 1, 2, 0, 0, 1]),
 array([1, 1, 1, 2, 0, 0, 0, 2, 0, 2, 2, 2, 1, 2]))

In [11]:
# 변환된 클래스명
le_weather.classes_, le_temperature.classes_

(array(['Overcast', 'Rainy', 'Sunny'], dtype=object),
 array(['Cool', 'Hot', 'Mild'], dtype=object))

In [12]:
# 입출력 구성
x_data = list(zip(weather_encoded, temperature_encoded))
y_data = df['play']

In [13]:
x_data, y_data

([(2, 1),
  (2, 1),
  (0, 1),
  (1, 2),
  (1, 0),
  (1, 0),
  (0, 0),
  (2, 2),
  (2, 0),
  (1, 2),
  (2, 2),
  (0, 2),
  (0, 1),
  (1, 2)],
 0      No
 1      No
 2     Yes
 3     Yes
 4     Yes
 5      No
 6     Yes
 7      No
 8     Yes
 9     Yes
 10    Yes
 11    Yes
 12    Yes
 13     No
 Name: play, dtype: object)

In [14]:
# 모델 생성
# feature 데이터가 문자열이기 때문에 LabelEncoding으로 숫자로 바꿔준다
model_nb = GaussianNB()
model_nb.fit(x_data, y_data)

GaussianNB()

In [15]:
# 예측
y_pred = model_nb.predict(x_data)
y_pred

array(['No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes',
       'No', 'Yes', 'Yes', 'Yes'], dtype='<U3')

In [16]:
# 성능 평가
print(classification_report(y_data, y_pred))

              precision    recall  f1-score   support

          No       0.60      0.60      0.60         5
         Yes       0.78      0.78      0.78         9

    accuracy                           0.71        14
   macro avg       0.69      0.69      0.69        14
weighted avg       0.71      0.71      0.71        14



In [17]:
le_weather.classes_, le_temperature.classes_

(array(['Overcast', 'Rainy', 'Sunny'], dtype=object),
 array(['Cool', 'Hot', 'Mild'], dtype=object))

In [18]:
# 'Rainy', 'Cool'한 경우 경기 여부 예측
a, b = le_weather.transform(['Rainy'])[0], le_temperature.transform(['Cool'])[0]
model_nb.predict([(a, b)])

array(['Yes'], dtype='<U3')

- precision: No라고 예측한 경우 60%가 실제 No였고 Yes라고 예측한 경우 78%가 실제 Yes였다
- recall: 실제 No에서 60%가 No로 예측되었고 실제 Yes에서 78%가 Yes로 예측되었다
- f1_score: precision과 recall의 조화 평균
- support: 데이터 개수 No 5개 Yes 9개 전체 14개
- accuracy: 정확도. 전체 학습데이터의 개수에서 각 클래스에서 자신의 클래스를 정확하게 맞춘 개수의 비율.
- macro avg: 단순평균
- weighted: 각 클래스에 속하는 표본의 갯수로 가중평균

#### make_column_transformer, OnehtoEncoder

In [19]:
# 입력 다시 정의
x_data = df.drop('play', axis=1)

In [20]:
# OneHotencoding할 컬럼명 설정
ct = make_column_transformer((OneHotEncoder(), x_data.columns))

In [21]:
# 변환
ct.fit_transform(df)

array([[0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.]])

In [22]:
# 변환한 데이터명
ct.get_feature_names()

['onehotencoder__x0_Overcast',
 'onehotencoder__x0_Rainy',
 'onehotencoder__x0_Sunny',
 'onehotencoder__x1_Cool',
 'onehotencoder__x1_Hot',
 'onehotencoder__x1_Mild']

In [23]:
# 파이프라인 생성
model_pipe = make_pipeline(ct, GaussianNB())
model_pipe.fit(x_data, y_data)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  Index(['weather', 'temperature'], dtype='object'))])),
                ('gaussiannb', GaussianNB())])

In [24]:
# 예측
y_pred = model_pipe.predict(x_data)
y_pred

array(['No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No',
       'Yes', 'Yes', 'No'], dtype='<U3')

In [25]:
print(classification_report(y_data, y_pred))

              precision    recall  f1-score   support

          No       0.50      1.00      0.67         5
         Yes       1.00      0.44      0.62         9

    accuracy                           0.64        14
   macro avg       0.75      0.72      0.64        14
weighted avg       0.82      0.64      0.63        14



In [26]:
# 'Rainy', 'Cool'한 경우 경기 여부 예측
# 입력한 데이터프레임과 컬럼명이 같은 데이터프레임을 생성해줘야 한다 
test_df = pd.DataFrame({'weather': ['Rainy'],
          'temperature': ['Cool']})
model_pipe.predict(test_df)

array(['No'], dtype='<U3')

문자열 데이터를 LabelEncoder로 변환해서 학습할 수도 있고 OneHotEncoder로 변환해서 학습할 수도 있다,
여기서는 LabelEncoder로 변환 후 학습하였을 때 f1_score가 좀 더 높게
나왔다

### MultinominalNB
- email ham, spam 데이터를 받아 예측

In [27]:
sms = pd.read_csv('data/spam.csv')
sms = sms[['v1', 'v2']]
sms

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [28]:
# 입출력 데이터
sms_data = sms[['v2']]
sms_label = sms['v1']

sms_data.shape, sms_label.shape

((5572, 1), (5572,))

In [29]:
from collections import Counter

In [30]:
sms_label.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [31]:
Counter(sms_label)

Counter({'ham': 4825, 'spam': 747})

#### 데이터 전처리를 위해 nltk 패키지 사용

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
from nltk.stem import PorterStemmer

In [33]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [34]:
# 구두점
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [47]:
text2 = ' '.join(''.join([' ' if ch[0] in string.punctuation else ch[0] for ch in sms_data.values]).split())
# [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
# [word.lower() for word in tokens]
tokens = [word.lower() for word in tokens]
    # 영어 불용어 
stopwds = stopwords.words('english')
tokens = [token for token in tokens if token not in stopwds]
    # 아주 짧은 단어는 대체로 중요한 의미를 갖지 않기 때문에 세글자 이상인 단어만 저장
tokens = [word for word in tokens if len(word) >= 3]

'go'

In [70]:
# 모든 전처리 과정 수행하기 위한 함수
def preprocessing(text):
    # 구두점 제거하고 단어별로 구분 후 한 문장으로 합치기
    text2 = ' '.join(''.join([' ' if ch in string.punctuation else ch for ch in text]).split())
    # 문장을 공백 기준으로 단어로 토큰화하고 다음 처리 단계를 위해 리스트에 모두 모은다
    # 문장으로 나누고 다시 문장을 단어로 나눈다
    tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
    # 모든 글자를 소문자로 통일해 말뭉치(corpus)에서의 중복을 피한다
    tokens = [word.lower() for word in tokens]
    # 영어 불용어 
    stopwds = stopwords.words('english')
    # 불용어 제거
    tokens = [token for token in tokens if token not in stopwds]
    # 아주 짧은 단어는 대체로 중요한 의미를 갖지 않기 때문에 세글자 이상인 단어만 저장
    tokens = [word for word in tokens if len(word) >= 3]
    # 불필요한 접미어 제거
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    # 표제어 처리를 위해 품사 등을 표시
    tagged_corpus = pos_tag(tokens)
    # 단어의 태그가 명사나 동사라면 단어의 형태를 명사나 동사로 바꿔주는 작업 후 다시 문자열로 묶는다
    pre_proc_text = " ".join([prat_lemmatize(token, tag) for token, tag in tagged_corpus])
    
    return pre_proc_text

In [71]:
# pos_tag 함수와 표제어 함수 입력값 사이의 불일치 때문에 일치시켜주는 함수
# 단어의 태그가 명사나 동사라면 단어의 형태를 명사나 동사로 바꿔주는 작업
Noun_tags = ['NN', 'NNP', 'NNPS', 'NNS']
Verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
lemmatizer = WordNetLemmatizer()

def prat_lemmatize(token, tag):
    # 태크가 명사라면
    if tag in Noun_tags:
        return lemmatizer.lemmatize(token, 'n')
    # 태그가 동사라면
    elif tag in Verb_tags:
        return lemmatizer.lemmatize(token, 'v')
    # 그 이외의 태그 
    else:
        return lemmatizer.lemmatize(token, 'n')

In [72]:
# 전처리 후 새로운 리스트에 담는다
sms_data2 = []
for i in sms_data.values:
    sms_data2.append(preprocessing(i[0]))

In [76]:
sms_data2

['jurong point crazi avail bugi great world buffet cine get amor wat',
 'lar joke wif oni',
 'free entri wkli comp win cup final tkt 21st may 2005 text 87121 receiv entri question std txt rate appli 08452810075over18',
 'dun say earli hor alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea �900 prize reward claim call 09061701461 claim code kl341 valid hour',
 'mobil month entitl updat latest colour mobil camera free call mobil updat free 08002986030',
 'gon home soon want talk stuff anymor tonight cri enough today',
 'six chanc win cash 100 000 pound txt csh11 send 87575 cost 150p day 6day tsandc appli repli info',
 'urgent week free membership �100 000 prize jackpot txt word claim 81010 www dbuk net lccltd pob

In [167]:
# 트레인 테스트셋 구성(7:3)
x_train, x_test, y_train, y_test = train_test_split(sms_data2, sms_label, test_size=0.3, stratify=sms_label, random_state=0)

In [168]:
len(sms_data2), len(x_train), len(x_test), y_train.shape, y_test.shape

(5572, 3900, 1672, (3900,), (1672,))

In [169]:
x_train

['yay final lol miss cinema trip last week',
 'well computerless time make oreo truffl',
 'think disturb',
 'watch hous ��� entertain ��� get whole hugh lauri thing ��� even stick ��� inde especi stick',
 'current food alon also',
 'txt call 86888 claim reward hour talk time use phone subscribe6gbp mnth inc 3hr stop txtstop www gamb',
 'great send account number',
 'addi amp art get home',
 'night end anoth day morn come special way may smile like sunni ray leav worri blue blue bay gud mrng',
 'eatin later eatin wif fren lei go home first',
 'plea call 08712402902 immedi urgent messag wait',
 'gibb unsold mike hussey',
 'januari male sale hot gay chat cheaper call 08709222922 nation rate min cheap min peak stop text call 08712460324 10p min',
 'meet soon princess ttyl',
 'thank fill complet calm reassur',
 'hey great deal farm tour 9am 5pm pax deposit may',
 'final match head toward draw predict',
 'pl stop bootydeli invit friend repli ye 434 434 see www sm bootydeli stop send stop frn

In [170]:
x_test

['way 2day normal way real uniqu hope know rest mylif hope find wot lose',
 'buy newspap alreadi',
 'part initi understand',
 'chikku yet free',
 'friend use call',
 'babe chloe smash saturday night great weekend miss visionsm com text stop stop 150p text',
 'plural noun research',
 'chanc might evapor soon violat privaci steal phone number employ paperwork cool plea contact report supervisor',
 'pub',
 'dont file bag work call tell find anyth room',
 'think bout drink tap spile seven pub ga broad canal',
 '',
 'love hear see sundayish',
 'oki thanx',
 'decemb mobil 11mth entitl updat latest colour camera mobil free call mobil updat vco free 08002986906',
 'guarante �1000 cash �2000 prize claim prize call custom servic repres 08714712379 10am 7pm cost 10p',
 'friend want drive someplac probabl take',
 'stupid possibl',
 'good morn plz call sir',
 'haf msn yiju hotmail com',
 'gon get taco',
 '1000 flirt txt girl bloke name age girl zoe 8007 join get chat',
 'also know lunch menu know',

In [179]:
# 단어를 벡터 형태로 바꾸고 TF-IDF(term frequency-inverse document frequency) 가중값을 수행한다
# 이 방법은 고빈도 단어에 관해서는 가중값을 높여주되, 이와 동시에 the, him, at 같은 일반 용어에 관해서는 가중값을 감점한다
# 가장 빈번한 단어 2000개로 제한해 사용
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english',
                            max_features=2392, strip_accents='unicode', norm='l2')

# 트레인 테스트 데이터에 관해 TF-IDF변환 수행
# todense 함수로 내용을 시각화하기 위해 데이터 생성
x_train2 = vectorizer.fit_transform(x_train).todense()
x_test2 = vectorizer.fit_transform(x_test).todense()

x_train2.shape, x_test2.shape

((3900, 2392), (1672, 2392))

In [180]:
# 모델 생성 및 학습
clf = MultinomialNB()
clf.fit(x_train2, y_train)

MultinomialNB()

In [181]:
# 예측
ytrain_pred = clf.predict(x_train2)
ytest_pred = clf.predict(x_test2)

ytrain_pred, ytest_pred

(array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4'),
 array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4'))

In [182]:
# train confusion matrix
pd.crosstab(y_train, ytrain_pred, rownames=['Actual'], colnames=['Predicted'])

Predicted,ham,spam
Actual,,
ham,3376,1
spam,79,444


In [183]:
# train score
print(classification_report(y_train, ytrain_pred))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      3377
        spam       1.00      0.85      0.92       523

    accuracy                           0.98      3900
   macro avg       0.99      0.92      0.95      3900
weighted avg       0.98      0.98      0.98      3900



In [184]:
# test confusion matrix
pd.crosstab(y_test, ytest_pred, rownames=['Actual'], colnames=['Predicted'])

Predicted,ham,spam
Actual,,
ham,1376,72
spam,139,85


In [185]:
# test score
print(classification_report(y_test, ytest_pred))

              precision    recall  f1-score   support

         ham       0.91      0.95      0.93      1448
        spam       0.54      0.38      0.45       224

    accuracy                           0.87      1672
   macro avg       0.72      0.66      0.69      1672
weighted avg       0.86      0.87      0.86      1672



트레인셋에서는 98%의 정확도로 매우 우수한 성능을 보였으나, 테스트셋에서는 87%의 정확도를 보였고 ham에 대한 f1_score값은 매우 높았으나 spam에 대한 f1_score값이 낮게 나왔다

In [196]:
clf.coef_, clf.intercept_

(array([[-6.67539471, -7.53342088, -7.37878623, ..., -8.37273472,
         -7.65024389, -7.95002126]]),
 array([-2.00915037]))

In [197]:
# 나이브 베이즈의 상위 10개 특징
feature_names = vectorizer.get_feature_names()
# 나이브 베이즈 가중값
coefs = clf.coef_
intercept = clf.intercept_
# 가중값과 단어를 묶고 오름차순 정렬
coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))

n = 10
coefs_with_fns[: n]

[(-8.372734718560167, '1000 claim'),
 (-8.372734718560167, '150p msg'),
 (-8.372734718560167, '21870000 mailbox'),
 (-8.372734718560167, '220'),
 (-8.372734718560167, '2nd attempt'),
 (-8.372734718560167, '2optout'),
 (-8.372734718560167, '2optout 087187262701'),
 (-8.372734718560167, '3qxj9'),
 (-8.372734718560167, '3qxj9 unsubscrib'),
 (-8.372734718560167, '500')]